As already mentioned in the report, here we are solving the following discrete problem


\begin{equation}
    \text{Find}\;\;\theta_h \in V_h\;\;\text{s.t.}\;\;a_h(\theta_h,v_h) + b_h(\theta_h,v_h)=l_h(v_h)\quad\forall v_h\in V_h\,.
\end{equation}


where $V_h = \mathbb{P}^r(\mathcal{T}_h)$ and $a_h$, $b_h$ and $l_h$ are the one defined in (2.6), (2.7) and (2.8) respectively.

In [1]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [2]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# First experiment - Exponential case
k = Constant(0.05)
u = Constant((1.0,1.0))
g = Constant(0)
t_exact = Expression('(exp((x[0] - 1) / k) - exp(-1 / k)) / (1 - exp(-1 / k)) + '
                     '(exp((x[1] - 1) / k) - exp(-1 / k)) / (1 - exp(-1 / k))', k=k, degree=2)

d = 100
degree = 1

In [ ]:
# Second experiment - Polynomial case
k = Constant(1e-4)
u = Expression(('x[1]','x[0]'), degree = 3)
g = Expression('-4*k + 4*x[0]*x[1]', k=k, degree = 3)
t_exact = Expression('pow(x[0],2)+pow(x[1],2)', degree = 3)

d = 100
degree = 2

In [3]:
# Third - trigonometric case
k = Constant(1e-4)
u = Expression(('pow(sin(x[1]),2)','pow(cos(x[0]),2)'), degree=4)
g = Expression('k*2*cos(x[0])*sin(x[1])-sin(x[0])*pow(sin(x[1]),3)+cos(x[1])*pow(cos(x[0]),3)', k=k, degree = 2)
t_exact = Expression('sin(x[1])*cos(x[0])', degree = 2)

d = 100
degree = 2

We want to clearify a little the notation used in the following code. In the definition of the linear form both "dx", "dS", and "ds" appears:

"dx": is the identifier of the integrals over the domain $\Omega$ \\
"dS": is the identifier of the integral over the internal facets, excluding the integrals over boundary facets \\
"ds": is the identifier of the integral over the boundary facets

The terms with ds in the left hand side come from the weak imposition of boundary conditions and since the code indentifies differently the two types of integral we were forced to separate the terms on the internal faces and the ones on the boundary facets.

In [4]:
def DiffAdv_DG(d, degree, k, g, u, t_exact):

  # 1. mesh generation
  mesh = UnitSquareMesh(d, d, 'crossed')

  # 2. definition of finite element space
  V = FunctionSpace(mesh, 'DG', degree)
  W = FunctionSpace(mesh, 'RT', 2)
  uh = project(u,W)
  # project on RT in order to keep the conformity

  # 3. defining test and trial function
  t = TrialFunction(V)
  v = TestFunction(V)
  n = FacetNormal(mesh)

  gamma = 9.1*degree**2
  h = CellDiameter(mesh)
  h_avg = (h('+') + h('-'))/2

  # 4. defining the bilinear form and the linear functional
  a = (k*dot(grad(t) , grad(v)))*dx \
    - (dot(avg(k*grad(t)) , jump(v,n)))*dS \
    - (dot(avg(k*grad(v)) , jump(t,n)))*dS \
    - (dot(k*grad(v) , n*t))*ds \
    + ((gamma/h_avg)*dot(jump(t,n) , jump(v,n)))*dS \
    + (t*v*gamma/h)*ds \
    - (dot(uh*t, grad(v)))*dx \
    + (dot(avg(uh*t) , jump(v,n)))*dS \
    + (abs(dot(uh('+'),n('+')))/2*dot(jump(t,n) , jump(v,n)))*dS \
    + 0.5*dot(uh,n)*t*v*ds \
    - 0.5*div(uh)*t*v*dx

  L = (g*v)*dx \
    - (dot(k*grad(v) , n*t_exact))*ds \
    + (t_exact*v*gamma/h)*ds \
    - 0.5*(dot(uh,n)*t_exact*v)*ds

  th = Function(V)
  solve(a == L, th)

  return th

Now we compute the errors and the orders of convergence

In [ ]:
# Convergence test
error_L2 = np.empty([1,4])
error_H1 = np.empty([1,4])
h = np.empty([1,4])

i = 0
for n in [20, 40, 80, 160]:
  t = DiffAdv_DG(n, degree, k, g, u, t_exact)
  error_L2[0,i] = errornorm(t_exact,t,'L2')
  error_H1[0,i] = errornorm(t_exact,t,'H1')
  h[0,i] = 1/n
  i = i+1

In [6]:
# rescaling of the errors
h = h / h[0,0]
error_L2 = error_L2 / error_L2[0,0]
error_H1 = error_H1 / error_H1[0,0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Plot the errors
plt.loglog(h[0,:], h[0,:], 'y', label='Order 1')
plt.loglog(h[0,:], h[0,:]**2, 'r', label='Order 2')
plt.loglog(h[0,:], error_L2[0,:], 'b--o', label='L2')
plt.loglog(h[0,:], error_H1[0,:], 'g--o', label='H1')

plt.title("Orders of convergence dG Transport Diffusion");
plt.xlabel("h")
plt.ylabel("Errors")
plt.legend(loc=4)

plt.show()

In [ ]:
# ORDERS of convergence
order_L2 =np.empty([1,3])
order_H1 =np.empty([1,3])

for j in [1,2,3]:
  order_L2[0,j-1]=math.log(error_L2[0,j-1]/error_L2[0,j],2)
  order_H1[0,j-1]=math.log(error_H1[0,j-1]/error_H1[0,j],2)

print(order_L2)
print(order_H1)